In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, shapiro, ttest_ind, boxcox, linregress
import plotly.express as px
from IPython.display import HTML, IFrame

ModuleNotFoundError: No module named 'pandas'

#### 1) Read in the `gapminder_clean.csv` data as a `pandas` `DataFrame`.

``` python
df = pd.read_csv("gapminder_clean.csv")
```

#### 2) Filter the data to include only rows where `Year` is `1962` and then make a scatter plot comparing `'CO2 emissions (metric tons per capita)'` and `gdpPercap for the filtered data.`

``` python
filt_df = df[df["Year"] == 1962]
co2_gdp_splt = sns.scatterplot(data = filt_df, x= "CO2 emissions (metric tons per capita)", y = "gdpPercap")
```

![Test](files/CO2_emissions_(metric_tons_per_capita)_vs._gdpPercap.png)

#### Preprocess by removing empty rows.
```python
# Remove rows if one or other has nas.
nas = filt_df["gdpPercap"].isna() | filt_df["CO2 emissions (metric tons per capita)"].isna()
filt_df = filt_df[~nas]

# Check that length of obs in each col is equal.
assert(len(filt_df["CO2 emissions (metric tons per capita)"]) == len(filt_df["gdpPercap"]))
```

#### 3) On the filtered data, calculate the pearson correlation of `CO2 emissions (metric tons per capita)` and `gdpPercap`. What is the Pearson R value and associated p value?

```python
# By-hand
# https://machinelearningmastery.com/how-to-use-correlation-to-understand-the-relationship-between-variables/
covariance = np.cov(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])
```
The second row and first column is the covariance between `CO2 emissions (metric tons per capita)` and `gdpPercap`.

```python
array([[2.46358644e+01, 4.48639240e+04],
       [4.48639240e+04, 9.52638497e+07]])
```

```python
p_corr_coeff = covariance[1,0] / (np.std(filt_df["CO2 emissions (metric tons per capita)"]) * np.std(filt_df["gdpPercap"]))
```

The pearson correlation between `'CO2 emissions (metric tons per capita)'` and `gdpPercap` can also be found using `scipy.stats`'s `pearsonr` function.

```python
# Using scipy.stats.pearsonr
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html
r, prob = pearsonr(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])
```

**The correlation coefficient (`r`) is ~0.926 with a probability (`prob`) of 1.129e-46.**
* This implies that there is a strong correlation between CO2 emissions per cap and gross domestic product per cap with a low probability of producing this r by random chance from uncorrelated data.

#### 4) On the unfiltered data, answer "In what year is the correlation between `'CO2 emissions (metric tons per capita)'` and `gdpPercap` the strongest?" Filter the dataset to that year for the next step...

```python
years = df["Year"].unique()

# create functions to repeat data processing and stat calculations
def remove_nas(df, col1, col2):
    nas = df[col1].isna() | df[col2].isna()
    filt_df = df[~nas]
    return filt_df

def filter_df_year(df, year):
    filt_df = df[df["Year"] == year]
    # remove rows if one or other has nas
    filt_df = remove_nas(filt_df, "CO2 emissions (metric tons per capita)", "gdpPercap")
    return filt_df

def yearly_correlation(df, year):
    filt_df = filter_df_year(df, year)
    r, prob = pearsonr(filt_df["CO2 emissions (metric tons per capita)"], filt_df["gdpPercap"])
    return r, prob

r_for_year = [yearly_correlation(df, year)[0] for year in years]

highest_r_year = years[list(r_for_year).index(max(r_for_year))]

```

**The correlation between `CO2 emissions (metric tons per capita)` and `gdpPercap` is strongest in `1967`.**

#### 5) Using plotly or bokeh, create an interactive scatter plot comparing `'CO2 emissions (metric tons per capita)'` and `gdpPercap`, where the point size is determined by `pop` (population) and the color is determined by the `continent`.

```python
df_h_r = filter_df_year(df, highest_r_year)

fig = px.scatter(df_h_r, 
                 x = df_h_r["CO2 emissions (metric tons per capita)"], y = df_h_r["gdpPercap"], 
                 size = df_h_r["pop"], color = df_h_r["continent"], 
                 title = "CO2 emissions (metric tons per capita) vs. gfpPercap")
```

In [2]:
IFrame(src='co2_em_gfppcap.html', width=800, height=600)

NameError: name 'IFrame' is not defined

#### 1. What is the relationship between `continent` and `'Energy use (kg of oil equivalent per capita)'`?

Here, a correlation coefficient between `continent` and `Energy use (kg of oil equivalent per capita)` was calculated.

First, any na values were removed from the dataset and `continent` was convertedto categorical codes.
```python
df_cnt_energy = remove_nas(df, "continent", "Energy use (kg of oil equivalent per capita)")

# convert continent column to categorical nums for tests
df_cnt_energy["continent"] = df_cnt_energy["continent"].astype("category").cat.codes
```

Next, normality of both columns was calculated using the Shapiro-Wilk test.
```python
# https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
# shapiro-wilk test checks for normality of sample data, H0 is that drawn from norm dist.
norm_test_cnt = shapiro(df_cnt_energy["continent"])
norm_test_energy = shapiro(df_cnt_energy["Energy use (kg of oil equivalent per capita)"])

cols_norm = True
alpha = 0.05
if norm_test_energy.pvalue > alpha and norm_test_cnt.pvalue > alpha:
    print("Fail to reject H0 for both columns. Both are normal.")
else:
    cols_norm = False
    print("Rejected H0 for a column. One column and/or other not normal.")
```

`cols_norm` outputs `False` and both of the columns have p-values below `alpha` (0.05); this means that the H0 can be rejected and that the data from the columns is normally distributed.

Knowing this, the Spearman's correlation (a non-parametric test) can be used to check if the two columns are correlated.

```python
if cols_norm:
    r, prob = pearsonr(df_cnt_energy["continent"], df_cnt_energy["Energy use (kg of oil equivalent per capita)"])
else:
    r, prob = spearmanr(df_cnt_energy["continent"], df_cnt_energy["Energy use (kg of oil equivalent per capita)"])

```

The Spearman's correlation coefficient (`r`) is `0.573` (`p = 3.289e-75`); **this suggests a positive, slightly high correlation between `continent` and `Energy use (kg of oul equivalent per capita`.**

#### 2) Is there a significant difference between Europe and Asia with respect to `Imports of goods and services (% of GDP)` in the years after 1990? (Stats test needed)

To check if there is a significant difference between Europe and Asia in `Imports of goods and services (% of GDP)` after the 1990s, a t-test (Student's or Welch's) was used for two samples.

First, the dataset was filtered taking only observations where the `Year` is greater than 1990. 

```python
df_post90s = df[df["Year"] > 1990]
```

Next, the dataset is cleaned of `na`s, filtered, and separated into two new dataframes: one where `continent` is Asia and the other where it is Europe.

```python
imp_col = "Imports of goods and services (% of GDP)"
df_post90s = remove_nas(df_post90s, "continent", imp_col)

df_post90s_asia = df_post90s[df_post90s["continent"] == "Asia"]
df_post90s_europe = df_post90s[df_post90s["continent"] == "Europe"]
```

Calculating the standard deviation (stdev) of both sample columns determines whether the Student's T-test or Welch's T-test should be used.
* If the **stdev of one column is double** that of the other, the **Welch's T-test** is used *(McDonald 130)*.
    * The Student's T-test assumes equal variance.
* **Otherwise, the Student's T-test** is used.

In this case, the standard deviation of `Imports of goods and services (% of GFP)` the filtered Asia dataset was twice (~2.003) that of the Europe dataset. With unequal variance, the Welch's T-test was used.
```python
p90s_asia_imp_std = np.std(df_post90s_asia[imp_col])
p90s_eur_imp_std = np.std(df_post90s_europe[imp_col])

# If stdev of one sample is two times larger than the other, then use Welch's t-test instead of Student's t-test
std_ratio = p90s_asia_imp_std / p90s_eur_imp_std

imp_cols = df_post90s_asia[imp_col], df_post90s_europe[imp_col]

if std_ratio < 2 or std_ratio > 0.5:
    # equal_var arg uses welch's t-test
    print("Using Welch's T-test.")
    t_stat, t_p = ttest_ind(*imp_cols, equal_var=False)
else:
    print("Using Student's T-test.")
    t_stat, t_p = ttest_ind(*imp_cols)
```

The resulting test **statistic was 1.355** and the **p-value was 0.178**. 
* As the p-value was above `alpha` of 0.05, the **H0 cannot be rejected and there is no significant difference between the means.** 

#### Works Cited
1. *McDonald, John H., Handbook of Biological Statistics, Sparky House Publishing, 2014*

#### 3) What is the country (or countries) that has the highest `Population density (people per sq. km of land area)` across all years? (i.e., which country has the highest average ranking in this category across each time point in the dataset?)

In this dataset, the years are 1967,1972, 1977, 1982, 1987, 1992, 1997, 2002, and 2007
```python
years = df["Year"].unique()
```

Group the dataset by `Year` and use the `agg` method to find the maximum index of `Population density (people per sq. km of land area)`.
```python
res = df.groupby(["Year"]).agg({f"{pop_den_col}":"idxmax"})
```

Then, access the dataset indices output from `res` and take the `Year` and `Country Name`.

```python
df.loc[res[pop_den_col]].loc[:, ["Year", "Country Name"]]
```

This yields the following years and associated countries.
```python
{1962: 'Monaco', 1967: 'Monaco', 1972: 'Macao SAR, China', 1977: 'Monaco', 1982: 'Monaco', 1987: 'Macao SAR, China', 1992: 'Macao SAR, China', 1997: 'Macao SAR, China', 2002: 'Macao SAR, China', 2007: 'Monaco'}
```

#### 4) What country (or countries) has shown the greatest increase in `Life expectancy at birth, total (years)` since 1962?

First, filter the dataset to `Year`s greater than 1962.
```python
df_post1962 = df[df["Year"] > 1962]
```

Group `df_post1962` by `Country Name` and apply `scipy.stats` linear regress function, `linregress` to the dataset using the columns `Year` and `Life expectancy at birth, total (years)`. 

```python
dle_dt = df_post1962.groupby("Country Name").apply(lambda x: linregress(x["Year"], x[life_exp_col]).slope)
```

Taking the slope attribute from the `linregress` object shows us how `Life expectancy at birth, total (years)` changes over `Year` since 1962.

```python
... linregress(x["Year"], x[life_exp_col]).slope
```

To check the country and its slope...
```python
dle_dt.idxmax(), dle_dt.loc[dle_dt.idxmax()]
```


**The country with the largest positive slope is Maldives with a slope of ~0.850; this means that the average life expectancy has increased by 0.850 years per year from 1967 to 2007.** 